In [25]:
import torch
import cvxpy as cp
import numpy as np
from cvxpylayers.torch import CvxpyLayer
from scipy.linalg import sqrtm
from scipy.linalg import solve_discrete_are

torch.set_default_dtype(torch.double)

In [26]:
np.random.seed(1)

# generate problem data
n, m = 8, 2
noise = np.sqrt(.25)
u_max = .1
Q0 = np.eye(n)
R0 = np.eye(m)
A = np.random.randn(n, n)
A /= np.max(np.abs(np.linalg.eig(A)[0]))
B = np.random.randn(n, m)
W = noise**2 * np.eye(n)

In [27]:
# compute lqr solution
P = cp.Variable((n, n), PSD=True)
R0cvxpy = cp.Parameter((m, m), PSD=True)

objective = cp.trace(P@W)
constraints = [cp.bmat([
    [R0cvxpy + B.T@P@B, B.T@P@A],
    [A.T@P@B, Q0+A.T@P@A-P]
]) >> 0, P >> 0]
R0cvxpy.value = R0
result = cp.Problem(cp.Maximize(objective), constraints).solve()
P_lqr = P.value
print(result)

6.037499214903595


In [28]:
P_are = solve_discrete_are(A, B, Q0, R0)
np.testing.assert_allclose(P_are, P_lqr, atol=1e-3)

In [29]:
# compute lower bound
P = cp.Variable((n, n), PSD=True)
R = cp.Variable((m, m), PSD=True)
lam = cp.Variable(m, nonneg=True)

objective = cp.trace(P@W) - (u_max**2)*cp.sum(lam)
constraints = [R - R0 << cp.diag(lam), P >> 0, R >> 0, lam >= 0]
constraints += [cp.bmat([
    [R + B.T@P@B, B.T@P@A],
    [A.T@P@B, Q0 + A.T@P@A-P]
]) >> 0]
result = cp.Problem(cp.Maximize(objective), constraints).solve()
P_lb = P.value
print(result)

10.794702792539413


In [30]:
K_lqr = -np.linalg.solve(R0 + B.T @ P_lqr @ B, B.T @ P_lqr @ A)

In [31]:
# set up policy
x = cp.Parameter((n, 1))
P_sqrt = cp.Parameter((n, n))
q = cp.Parameter(n)

u = cp.Variable((m, 1))
xnext = cp.Variable((n, 1))

objective = cp.quad_form(u, R0) + cp.sum_squares(P_sqrt @ xnext) + q @ xnext
constraints = [xnext == A @ x + B @ u, cp.norm(u, "inf") <= u_max]
prob = cp.Problem(cp.Minimize(objective), constraints)
policy = CvxpyLayer(prob, [x, P_sqrt, q], [u])

In [32]:
Qt, Rt, At, Bt = map(torch.from_numpy, [Q0, R0, A, B])

In [33]:
def loss(time_horizon, batch_size, K, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    x_batch = noise * torch.randn(batch_size, n, 1).double()
    K_batch = K.repeat(batch_size, 1, 1)
    Qt_batch = Qt.repeat(batch_size, 1, 1)
    Rt_batch = Rt.repeat(batch_size, 1, 1)
    At_batch = At.repeat(batch_size, 1, 1)
    Bt_batch = Bt.repeat(batch_size, 1, 1)
    loss = 0.0
    for _ in range(time_horizon):
        u_batch = torch.clamp(K @ x_batch, min=-u_max, max=u_max)
        state_cost = torch.bmm(torch.bmm(Qt_batch, x_batch).transpose(2, 1), x_batch)
        control_cost = torch.bmm(torch.bmm(Rt_batch, u_batch).transpose(2, 1), u_batch)
        cost_batch = (state_cost.squeeze() + control_cost.squeeze())
        loss += cost_batch.sum() / (time_horizon * batch_size)
        x_batch = torch.bmm(At_batch, x_batch) + \
            torch.bmm(Bt_batch, u_batch) + \
            noise * torch.randn(batch_size, n, 1).double()
    return loss

In [34]:
K_clipped = loss(100, 6, torch.from_numpy(K_lqr), seed=0).item()

In [35]:
def loss(time_horizon, batch_size, P_sqrt, q, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    x_batch = noise * torch.randn(batch_size, n, 1).double()
    P_sqrt_batch = P_sqrt.repeat(batch_size, 1, 1)
    q_batch = q.repeat(batch_size, 1)
    Qt_batch = Qt.repeat(batch_size, 1, 1)
    Rt_batch = Rt.repeat(batch_size, 1, 1)
    At_batch = At.repeat(batch_size, 1, 1)
    Bt_batch = Bt.repeat(batch_size, 1, 1)
    loss = 0.0
    for _ in range(time_horizon):
        u_batch, = policy(x_batch, P_sqrt_batch, q_batch, solver_args={"acceleration_lookback": 0, "eps":1e-8, "max_iters":10000})
        state_cost = torch.bmm(torch.bmm(Qt_batch, x_batch).transpose(2, 1), x_batch)
        control_cost = torch.bmm(torch.bmm(Rt_batch, u_batch).transpose(2, 1), u_batch)
        cost_batch = (state_cost.squeeze() + control_cost.squeeze())
        loss += cost_batch.sum() / (time_horizon * batch_size)
        x_batch = torch.bmm(At_batch, x_batch) + \
            torch.bmm(Bt_batch, u_batch) + \
            noise * torch.randn(batch_size, n, 1).double()
    return loss

In [36]:
with torch.no_grad():
    clf_lqr = loss(100, 6, torch.from_numpy(sqrtm(P_lqr)), torch.zeros(n, dtype=torch.double), seed=0).item()
    clf_lb = loss(100, 6, torch.from_numpy(sqrtm(P_lb)), torch.zeros(n, dtype=torch.double), seed=0).item()

/home/shane/miniconda3/envs/cvxpylayers/lib/python3.7/site-packages/diffcp/cone_program.py:259: UserWarning: Solved/Inaccurate.
  warnings.warn("Solved/Inaccurate.")


In [37]:
clf_lqr, clf_lb

(16.940040755077668, 12.975429366507662)

In [38]:
# initialize parameters
torch.manual_seed(0)
P_sqrt = torch.from_numpy(sqrtm(P_lqr)); P_sqrt.requires_grad_(True);
q = torch.zeros(n).double(); q.requires_grad_(True);

In [39]:
opt = torch.optim.SGD([P_sqrt, q], lr=.1)
losses = []
for k in range(100):
    with torch.no_grad():
        test_loss = loss(100, 6, P_sqrt.detach(), q.detach(), seed=0).item()
        losses.append(test_loss)
        P_np = (P_sqrt.t() @ P_sqrt).detach().numpy()
        dist = np.linalg.norm(P_np - P_lb)
        print("it: %03d, loss: %3.3f, dist: %3.3f" % (k+1, test_loss, dist))
    opt.zero_grad()
    l = loss(100, 6, P_sqrt, q, seed=k+1)
    l.backward()
    opt.step()
    if k == 50:
        opt = torch.optim.SGD([P_sqrt, q], lr=.01)

it: 001, loss: 16.940, dist: 20.232
it: 002, loss: 14.258, dist: 18.828
it: 003, loss: 14.035, dist: 18.613
it: 004, loss: 13.795, dist: 18.323
it: 005, loss: 13.677, dist: 17.990
it: 006, loss: 13.436, dist: 17.716
it: 007, loss: 13.311, dist: 17.600
it: 008, loss: 13.265, dist: 17.450
it: 009, loss: 13.331, dist: 17.374
it: 010, loss: 13.231, dist: 17.224
it: 011, loss: 13.333, dist: 17.065
it: 012, loss: 13.397, dist: 17.053
it: 013, loss: 13.474, dist: 17.010
it: 014, loss: 13.248, dist: 16.918
it: 015, loss: 13.383, dist: 17.065
it: 016, loss: 13.296, dist: 17.016
it: 017, loss: 13.347, dist: 16.944
it: 018, loss: 13.295, dist: 16.676
it: 019, loss: 13.265, dist: 16.674
it: 020, loss: 13.175, dist: 16.662
it: 021, loss: 13.159, dist: 16.625
it: 022, loss: 13.211, dist: 16.636
it: 023, loss: 13.231, dist: 16.481
it: 024, loss: 13.234, dist: 16.443
it: 025, loss: 13.150, dist: 16.425
it: 026, loss: 13.065, dist: 16.363
it: 027, loss: 13.053, dist: 16.355
it: 028, loss: 13.062, dist:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from latexify import latexify

latexify(fig_width=4.5)
plt.semilogy(losses, c='k', label='COCP')
plt.gca().yaxis.set_minor_formatter(matplotlib.ticker.ScalarFormatter())
plt.axhline(clf_lb, linestyle='--', c='k', label='upper bound')
plt.axhline(result, linestyle='-.', c='k', label='lower bound')
plt.xlabel('iteration')
plt.ylabel('cost')
plt.subplots_adjust(left=.15, bottom=.2)
plt.ylim(10, 18)
plt.legend(loc='upper right')
plt.savefig("lqr_constrained.pdf")
plt.show()

## Neural net

In [118]:
from torch import nn
from torch.nn import functional

class Net(nn.Module):
    def __init__(self, n, K):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, m)
        self.fc_K = nn.Linear(n, m, bias=False)
        self.fc_K.weight.data = K.data
#         for fc in [self.fc1, self.fc2, self.fc3]:
#             fc.weight.data.zero_()
#             fc.bias.data.zero_()

    def forward(self, x):
        u = self.fc_K(x) 
        x = functional.relu(self.fc1(x))
        x = functional.relu(self.fc2(x))
        x = torch.clamp(self.fc3(x), -u_max, u_max)
        return x

In [124]:
policy = Net(n, torch.from_numpy(K_lqr))

In [125]:
def loss(time_horizon, batch_size, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    x_batch = noise * torch.randn(batch_size, n, 1).double()
    Qt_batch = Qt.repeat(batch_size, 1, 1)
    Rt_batch = Rt.repeat(batch_size, 1, 1)
    At_batch = At.repeat(batch_size, 1, 1)
    Bt_batch = Bt.repeat(batch_size, 1, 1)
    loss = 0.0
    for _ in range(time_horizon):
        u_batch = policy(x_batch.squeeze(-1))
        u_batch = u_batch.unsqueeze(-1)
        state_cost = torch.bmm(torch.bmm(Qt_batch, x_batch).transpose(2, 1), x_batch)
        control_cost = torch.bmm(torch.bmm(Rt_batch, u_batch).transpose(2, 1), u_batch)
        cost_batch = (state_cost.squeeze() + control_cost.squeeze())
        loss += cost_batch.sum() / (time_horizon * batch_size)
        x_batch = torch.bmm(At_batch, x_batch) + \
            torch.bmm(Bt_batch, u_batch) + \
            noise * torch.randn(batch_size, n, 1).double()
    return loss

In [128]:
opt = torch.optim.Adam(policy.parameters(), lr=3e-4)
losses = []
for k in range(10000):
    with torch.no_grad():
        test_loss = loss(100, 6, seed=0).item()
        losses.append(test_loss)
        print("it: %03d, loss: %3.3f" % (k+1, test_loss))
    opt.zero_grad()
    l = loss(100, 6, seed=k+1)
    l.backward()
    opt.step()
#     if k == 200:
#         opt = torch.optim.SGD(policy.parameters(), lr=1e-5)

it: 001, loss: 13.605
it: 002, loss: 13.605
it: 003, loss: 13.604
it: 004, loss: 13.596
it: 005, loss: 13.575
it: 006, loss: 13.551
it: 007, loss: 13.527
it: 008, loss: 13.505
it: 009, loss: 13.501
it: 010, loss: 13.491
it: 011, loss: 13.489
it: 012, loss: 13.482
it: 013, loss: 13.471
it: 014, loss: 13.464
it: 015, loss: 13.456
it: 016, loss: 13.448
it: 017, loss: 13.440
it: 018, loss: 13.431
it: 019, loss: 13.415
it: 020, loss: 13.397
it: 021, loss: 13.379
it: 022, loss: 13.360
it: 023, loss: 13.343
it: 024, loss: 13.330
it: 025, loss: 13.321
it: 026, loss: 13.313
it: 027, loss: 13.309
it: 028, loss: 13.304
it: 029, loss: 13.297
it: 030, loss: 13.293
it: 031, loss: 13.289
it: 032, loss: 13.283
it: 033, loss: 13.279
it: 034, loss: 13.275
it: 035, loss: 13.273
it: 036, loss: 13.275
it: 037, loss: 13.276
it: 038, loss: 13.279
it: 039, loss: 13.280
it: 040, loss: 13.280
it: 041, loss: 13.278
it: 042, loss: 13.278
it: 043, loss: 13.277
it: 044, loss: 13.276
it: 045, loss: 13.272
it: 046, l

KeyboardInterrupt: 

In [ ]:
plt.plot(losses)

## Piecewise quadratic

In [55]:
K = 25

In [56]:
# set up policy
x = cp.Parameter(n)
P_sqrt_cp = [cp.Parameter((n, n), PSD=True) for _ in range(K)]
q_cp = [cp.Parameter(n) for _ in range(K)]
r_cp = [cp.Parameter(1) for _ in range(K)]

u = cp.Variable(m)
xnext = cp.Variable(n)
t = cp.Variable(1)

objective = cp.quad_form(u, R0) + t
constraints = [xnext == A @ x + B @ u, cp.norm(u, "inf") <= u_max]
constraints += [cp.sum_squares(P_sqrt_cp[i] @ xnext) + q_cp[i] @ xnext + r_cp[i] <= t for i in range(K)]
prob = cp.Problem(cp.Minimize(objective), constraints)
policy = CvxpyLayer(prob, [x] + P_sqrt_cp + q_cp + r_cp, [u])

In [57]:
P_sqrt = [torch.from_numpy(sqrtm(P_lqr) + 1e-3*np.random.randn(n, n)) for _ in range(K)]
for i in range(K):
    P_sqrt[i].requires_grad_(True)
q = [torch.randn(n, requires_grad=True) for _ in range(K)]
r = [torch.zeros(1, requires_grad=True) for _ in range(K)]

In [58]:
def loss(time_horizon, batch_size, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    x_batch = noise * torch.randn(batch_size, n).double()
    Qt_batch = Qt.repeat(batch_size, 1, 1)
    Rt_batch = Rt.repeat(batch_size, 1, 1)
    At_batch = At.repeat(batch_size, 1, 1)
    Bt_batch = Bt.repeat(batch_size, 1, 1)
    loss = 0.0
    for _ in range(time_horizon):
        inputs = [x_batch] + P_sqrt + q + r
        u_batch, = policy(*inputs, solver_args={"acceleration_lookback": 0, "eps":1e-5, "max_iters":10000})
        state_cost = torch.bmm(torch.bmm(Qt_batch, x_batch.unsqueeze(-1)).transpose(2, 1), x_batch.unsqueeze(-1))
        control_cost = torch.bmm(torch.bmm(Rt_batch, u_batch.unsqueeze(-1)).transpose(2, 1), u_batch.unsqueeze(-1))
        cost_batch = (state_cost.squeeze() + control_cost.squeeze())
        loss += cost_batch.sum() / (time_horizon * batch_size)
        x_batch = torch.bmm(At_batch, x_batch.unsqueeze(-1)) + \
            torch.bmm(Bt_batch, u_batch.unsqueeze(-1)) + \
            noise * torch.randn(batch_size, n, 1).double()
        x_batch = x_batch.squeeze(-1)
    return loss

In [59]:
def norm(list_of_pytorch_tensors):
    with torch.no_grad():
        return torch.cat([t.view(-1) for t in list_of_pytorch_tensors]).norm().item()

In [63]:
params = P_sqrt + q + r
opt = torch.optim.SGD(params, lr=0.1)
losses = []
for k in range(100):
    with torch.no_grad():
        test_loss = loss(100, 6, seed=0).item()
        losses.append(test_loss)
        if params[0].grad is None:
            grad_norm = np.nan
        else:
            grad_norm = norm([p.grad.data for p in params])
        print("it: %03d, loss: %3.3f, grad_norm: %3.3f" % (k+1, test_loss, grad_norm))
    opt.zero_grad()
    l = loss(100, 6, seed=k+1)
    l.backward()
    torch.nn.utils.clip_grad_norm_(params, 10)
    opt.step()
    if k == 50:
        opt = torch.optim.SGD(params, lr=0.005)

it: 001, loss: 13.368, grad_norm: 0.000
it: 002, loss: 13.218, grad_norm: 2.971
it: 003, loss: 13.293, grad_norm: 2.776
it: 004, loss: 13.455, grad_norm: 5.518
it: 005, loss: 13.715, grad_norm: 3.672
it: 006, loss: 13.730, grad_norm: 3.191
it: 007, loss: 13.817, grad_norm: 3.788
it: 008, loss: 13.735, grad_norm: 4.652
it: 009, loss: 13.792, grad_norm: 4.192
it: 010, loss: 13.480, grad_norm: 4.183
it: 011, loss: 13.299, grad_norm: 6.466
it: 012, loss: 13.519, grad_norm: 3.917
it: 013, loss: 13.386, grad_norm: 4.412
it: 014, loss: 13.521, grad_norm: 2.561
it: 015, loss: 13.845, grad_norm: 3.820
it: 016, loss: 13.671, grad_norm: 1.890
it: 017, loss: 13.580, grad_norm: 2.818
it: 018, loss: 13.508, grad_norm: 6.822
it: 019, loss: 13.659, grad_norm: 8.734
it: 020, loss: 13.598, grad_norm: 3.174
it: 021, loss: 13.446, grad_norm: 4.614
it: 022, loss: 13.518, grad_norm: 4.279
it: 023, loss: 13.473, grad_norm: 5.674
it: 024, loss: 13.466, grad_norm: 3.205
it: 025, loss: 13.397, grad_norm: 4.298


NameError: name 'param' is not defined